In [70]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [71]:
path_deg = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/res_deg.csv'
path_mir = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results.csv'
path_mir_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/all_results_targets.csv'


In [73]:
df_mir = pd.read_csv(path_mir, sep='\t')
total_mir = len(df_mir.index)
print(total_mir)
df_mir[df_mir.MITE.notnull()].head(2)

260


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,22,23,24,seqname,position,start,end,MITE,mirbase,annotation
5,1B:161992200-161992280,sun_all_Cluster_8835,81,135,1.069,5,0.919,+,UCUGUUCACAAAUGUAAGACG,106,...,0,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron
9,1B:565664575-565664794,mrcv_all_Cluster_2513,220,223,1.750,1,0.991,+,CAUUUUCCUAUAGACUUGGUC,142,...,7,0,0,1B,565664575-565664794,565664575,565664794,"MITE_1151,MITE_1572,MITE_1565,MITE_606,MITE_68...",NaN,intron


In [77]:
df_deg = pd.read_csv(path_deg, sep=',')
print(len(df_deg.index))
df_deg.head(2)

3128


,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,MITE,annotation
0,TraesCS1A02G000300.1:66,sun_all_Cluster_118867,TraesCS1A02G000300.1,54,75,66,-43.6,-28.4,0.651376,9.0,"1-2,75-74;5-11,71-65;13-14,63-62;16-22,60-54","3-4,73-72[SIL];12-12,64-64[SIL];15-15,61-61[SIL]",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.951576,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN,five_prime_UTR
1,TraesCS1A02G000400.1:1381,sun_all_Cluster_118867,TraesCS1A02G000400.1,1369,1390,1381,-43.6,-28.4,0.651376,9.0,"1-2,1390-1389;5-11,1386-1380;13-14,1378-1377;1...","3-4,1388-1387[SIL];12-12,1379-1379[SIL];15-15,...",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.955557,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN,NaN


In [79]:
has_target = 0
df_mir['has_target'] = 0
has_same_target = 0
for k,v in df_mir.iterrows():
    df_target = df_deg[df_deg.Query == v.Name]
    if len(df_target.index) > 0:
        has_target += 1
        if df_mir.at[k, 'has_target'] == 0:
            df_mir.at[k, 'has_target'] = 1
        if not pd.isna(v.MITE):
            prods = v.MITE.split(',')
            for k2,v2 in df_target.iterrows():
                if not pd.isna(v2.MITE):
                    target = v2.MITE.split(',')
                    prods = set(filter(None, prods))
                    target = set(filter(None, target))
                    inter = target.intersection(prods)
                    if inter:
                        has_same_target += 1
                        print(v2.SiteID, v.Name, inter)
                        df_mir.at[k, 'has_target'] = 2

TraesCS2D02G327200.1:2349 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G102200.1:1506 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS3B02G231700.2:1224 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5B02G480100.1:1530 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS5D02G261800.1:1451 sun_all_Cluster_8835 {'MITE_1157'}
TraesCS7A02G218800.1:1394 mrcv_all_Cluster_13743 {'MITE_867'}
TraesCS2A02G274200.2:2583 sun_all_Cluster_136100 {'MITE_376'}
TraesCS2D02G578900.1:1854 sun_all_Cluster_136100 {'MITE_698', 'MITE_376', 'MITE_1128', 'MITE_975', 'MITE_1579'}


In [80]:
total_mir

260

In [81]:
has_target

210

In [82]:
has_target * 100 / total_mir

80.76923076923077

In [83]:
has_same_target

8

In [84]:
len(df_mir[df_mir.has_target == 2].index) * 100 / total_mir

1.1538461538461537

In [88]:
df_deg.head(2)

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,MITE,annotation
0,TraesCS1A02G000300.1:66,sun_all_Cluster_118867,TraesCS1A02G000300.1,54,75,66,-43.6,-28.4,0.651376,9.0,"1-2,75-74;5-11,71-65;13-14,63-62;16-22,60-54","3-4,73-72[SIL];12-12,64-64[SIL];15-15,61-61[SIL]",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.951576,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN,five_prime_UTR
1,TraesCS1A02G000400.1:1381,sun_all_Cluster_118867,TraesCS1A02G000400.1,1369,1390,1381,-43.6,-28.4,0.651376,9.0,"1-2,1390-1389;5-11,1386-1380;13-14,1378-1377;1...","3-4,1388-1387[SIL];12-12,1379-1379[SIL];15-15,...",(((((((.((.(((((((..((&))..))))))).)).))))))),GCAGAUCUUGGUGGUAGUAGCA&UGAAGCUGCCAGCAUGAUCUGC,3,0.955557,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN,NaN


In [111]:
df_mir[(df_mir.has_target!=0) & (df_mir.MITE.notnull())]

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,23,24,seqname,position,start,end,MITE,mirbase,annotation,has_target
5,1B:161992200-161992280,sun_all_Cluster_8835,81,135,1.069,5,0.919,+,UCUGUUCACAAAUGUAAGACG,106,...,0,2,1B,161992200-161992280,161992200,161992280,"MITE_1712,MITE_233,MITE_1717,MITE_1157",hvu-miR6197,intron,2
9,1B:565664575-565664794,mrcv_all_Cluster_2513,220,223,1.750,1,0.991,+,CAUUUUCCUAUAGACUUGGUC,142,...,0,0,1B,565664575-565664794,565664575,565664794,"MITE_1151,MITE_1572,MITE_1565,MITE_606,MITE_68...",NaN,intron,1
30,2B:5671327-5671460,sun_all_Cluster_28681,134,562,4.450,504,0.016,-,UAUCUGGACAAAUCUGAGACA,293,...,2,4,2B,5671327-5671460,5671327,5671460,"MITE_798,MITE_798,MITE_148,MITE_148,DTT_Hvul_P...",NaN,intron,1
37,2B:482720682-482720772,sun_all_Cluster_33132,91,102,0.808,91,0.078,-,AUCUUCUAUCGUGGGACGAAG,48,...,1,1,2B,482720682-482720772,482720682,482720772,"MITE_1444,MITE_1444,MITE_1030,MITE_1030,DTT_Ta...",NaN,intron,1
44,2B:720801630-720801713,mrcv_all_Cluster_8534,84,505,3.963,486,1.000,+,AUUGAACUAAGGAGGGGUGGA,386,...,0,0,2B,720801630-720801713,720801630,720801713,"MITE_1643,MITE_1643",NaN,intron,1
62,3A:746087222-746087306,mrcv_all_Cluster_12767,85,181,1.421,16,0.000,-,UAAUAUGGAUUGGAGGGGGUA,74,...,5,13,3A,746087222-746087306,746087222,746087306,MITE_660,oan-miR-1400-3p,"three_prime_UTR, exon",1
70,3B:587564305-587564383,mrcv_all_Cluster_13743,79,76,0.596,68,1.000,+,AUUGUGUACAGAGGGAGUAGU,67,...,0,0,3B,587564305-587564383,587564305,587564383,MITE_867,NaN,intron,2
107,4D:142764794-142764971,sun_all_Cluster_82408,178,77,0.610,5,0.831,+,UAGCUUAUUCUACAUCCCAGU,24,...,0,11,4D,142764794-142764971,142764794,142764971,"MITE_45,MITE_45,MITE_45,MITE_490,MITE_490,MITE...",NaN,intergenic,1
234,7A:668530089-668530164,sun_all_Cluster_136100,76,96,0.760,2,0.927,+,CUUCUGAUUUACUCGUCGUGG,26,...,0,26,7A,668530089-668530164,668530089,668530164,"MITE_1128,MITE_1128,MITE_975,MITE_975,MITE_698...",NaN,intergenic,2
243,7B:223173899-223174050,mrcv_all_Cluster_34329,152,80,0.628,2,1.000,+,UGGCAAAUCUAGUUGGUGAGC,44,...,0,1,7B,223173899-223174050,223173899,223174050,MITE_1013,NaN,intron,1


In [113]:
df_deg[(df_deg.DegradomeCategory == 0) & (df_deg.Query == 'sun_all_Cluster_28681')]

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path,lib,MITE,annotation
1376,TraesCS4A02G085100.1:554,sun_all_Cluster_28681,TraesCS4A02G085100.1,543,563,554,-34.6,-23.0,0.66474,6.0,"1-17,563-547","18-21,546-543[UP3]",....(((((((((((((((((&)))))))))))))))))....,ACAUUUGGGUUUGUCCAGAUG&UAUCUGGACAAAUCUGAGACA,0,0.075452,/home/juan/Desktop/juan/bio/mirna_mite/data/re...,SRR1197125,NaN,three_prime_UTR


In [114]:
df_ann[df_ann.transcript=='TraesCS4A02G085100.1']

,seqname,source,feature,start,end,score,strand,frame,attribute,transcript
1179969,4A,IWGSC,five_prime_UTR,89896035,89896176,.,+,.,Parent=transcript:TraesCS4A02G085100.1,TraesCS4A02G085100.1
1179970,4A,IWGSC,exon,89896035,89896231,.,+,.,Parent=transcript:TraesCS4A02G085100.1;Name=Tr...,TraesCS4A02G085100.1
1179972,4A,IWGSC,intron,89896232,89896706,0,+,.,Parent=transcript:TraesCS4A02G085100.1;Name=Tr...,TraesCS4A02G085100.1
1179973,4A,IWGSC,exon,89896707,89896854,.,+,.,Parent=transcript:TraesCS4A02G085100.1;Name=Tr...,TraesCS4A02G085100.1
1179976,4A,IWGSC,intron,89896855,89897808,0,+,.,Parent=transcript:TraesCS4A02G085100.1;Name=Tr...,TraesCS4A02G085100.1
1179977,4A,IWGSC,exon,89897809,89898255,.,+,.,Parent=transcript:TraesCS4A02G085100.1;Name=Tr...,TraesCS4A02G085100.1
1179978,4A,IWGSC,three_prime_UTR,89897885,89898255,.,+,.,Parent=transcript:TraesCS4A02G085100.1,TraesCS4A02G085100.1


In [85]:
df_mir.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,23,24,seqname,position,start,end,MITE,mirbase,annotation,has_target
0,1A:309399489-309399664,mrcv_all_Cluster_872,176,553,4.340,1,0.002,-,ACGGCAUAGAGGCACUGCAAA,327,...,5,4,1A,309399489-309399664,309399489,309399664,NaN,NaN,intergenic,1
1,1A:309399491-309399581,sun_all_Cluster_2789,91,459,3.635,4,0.007,-,ACGGCAUAGAGGCACUGCAAA,196,...,0,5,1A,309399491-309399581,309399491,309399581,NaN,NaN,intergenic,1


In [ ]:
#mirna mite prod -> target region feature

In [102]:
mirs_mites = df_mir[df_mir.MITE.notnull()].Name.unique()

In [103]:
df_deg['annotation'].fillna('exon', inplace=True)


In [104]:
df_deg_comes_mites = df_deg[df_deg.Query.isin(mirs_mites)]

In [105]:
df_deg.annotation.value_counts()

exon               2139
three_prime_UTR     736
five_prime_UTR      253
Name: annotation, dtype: int64

In [106]:
df_deg_comes_mites.annotation.value_counts()

exon               160
three_prime_UTR    119
five_prime_UTR      44
Name: annotation, dtype: int64

In [107]:
df_deg[df_deg.MITE.notnull()].annotation.value_counts()

three_prime_UTR    59
exon               54
five_prime_UTR     21
Name: annotation, dtype: int64

In [56]:
df_mir.to_csv(path_mir_res, sep='\t', index=None)